In [83]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import time

In [84]:
def get_links(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    test = soup.find(id='div_sched_2020-2021_9_1')
    x = "score"
    test = test.findAll("td", {"data-stat": x})
    link_list = []
    for i in test:
        i = i.find("a")
        if i is not None:
         link = i['href']
         link_list.append(link)
         #print(link)
    return link_list

In [85]:
#@title

#stats matchday
statsm = ["player", "shirtnumber", "nationality", "position", "age", "minutes", "goals", "assists", "touches", "sca",
          "gca", 'pens_made', 'shots_total', 'shots_on_target', 'pressures', 'tackles', 'interceptions', 'blocks', 'xg',
          'xa', 'passes', 'passes_pct', 'progressive_passes', 'progressive_carries', 'dribbles_completed', 'dribbles']

#stats keeper matchday
statskeepers = ['player', 'minutes','age' , 'gk_shots_on_target_against', 'gk_goals_against', 'gk_saves', 'gk_save_pct', 'gk_psxg', 'gk_passes_completed_launched', 'gk_passes', 'gk_passes_throws', 'gk_pct_passes_launched', 'gk_def_actions_outside_pen_area' ]

keepers = ["player","nationality","position","squad","age","birth_year","games_gk","games_starts_gk","minutes_gk","goals_against_gk","goals_against_per90_gk","shots_on_target_against","saves","save_pct","wins_gk","draws_gk","losses_gk","clean_sheets","clean_sheets_pct","pens_att_gk","pens_allowed","pens_saved","pens_missed_gk"]
keepers3 = ["players_used","games_gk","games_starts_gk","minutes_gk","goals_against_gk","goals_against_per90_gk","shots_on_target_against","saves","save_pct","wins_gk","draws_gk","losses_gk","clean_sheets","clean_sheets_pct","pens_att_gk","pens_allowed","pens_saved","pens_missed_gk"]
#advance goalkeeping(keepersadv)


#Functions to get the data in a dataframe using BeautifulSoup

def get_tables_m(url):
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", res.text), 'lxml')
    all_tables = soup.findAll("tbody")
    matchweek = soup.find(id='content').find("div").contents[1]
    date = soup.find("div", {"class": "scorebox_meta"}).find("div").contents[0].contents[0]
    date = date['href']
    date = date[12:22]
    #date = soup.find("div", {"class": "scorebox_meta"}).soup.find("span", {"class": "venuetime"})


    team_H = soup.findAll("strong")[2].find("a").contents[0]
    if soup.findAll("strong")[5].find("a") is None:
        team_A = soup.findAll("strong")[4].find("a").contents[0]
    else:
        team_A = soup.findAll("strong")[5].find("a").contents[0]

    team1_table = all_tables[0]
    team2_table = all_tables[7]
    gk1_table = all_tables[6]
    gk2_table = all_tables[13]


    #gk1_table.append(gk2_table)
    #team1_table.append(team2_table)

    return team1_table, team2_table, gk1_table, gk2_table, matchweek, date ,team_H, team_A


def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    rows_player
    #print(rows_player)
    for row in rows_player:
        #print(row)
        if (row.find('th', {"scope": "row"}) != None):

            for f in features_wanted_player:
                #print(f)
                cell = row.find("td", {"data-stat": f})
                #print(cell)
                if cell is None:
                    text = row.find('th', {"scope": "row"}).find('a').contents[0]
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
                #print(cell)
                else:
                    a = cell.text.strip().encode()
                    text = a.decode("utf-8")
                    #print(text)
                    if (text == ''):
                        text = '0'
                    if ((f != 'player') & (f != 'nationality') & (f != 'position') & (f != 'squad') & (f != 'age') & (
                            f != 'birth_year')):
                        text = float(text.replace(',', ''))
                    if f in pre_df_player:
                        pre_df_player[f].append(text)
                    else:
                        pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player




def frame_for_category(category, top, end, features):
    url = (top + category + end)
    print(url)
    player_table_H, player_table_A, gk_table_H, gk_table_A, matchweek, date ,team_H, team_A = get_tables_m(url)
    #print(player_table)
    df_player_H = get_frame(features, player_table_H)
    df_player_H['Team'] = team_H
    df_player_A = get_frame(features, player_table_A)
    df_player_A['Team'] = team_A
    df_player = pd.concat([df_player_H, df_player_A])
    df_player['matchweek'] = matchweek
    df_player['date'] = date
    df_gk_H = get_frame(statskeepers, gk_table_H)
    df_gk_H['Team'] = team_H
    df_gk_A = get_frame(statskeepers, gk_table_A)
    df_gk_A['Team'] = team_A
    df_gk = pd.concat([df_gk_H, df_gk_A])
    df_gk['matchweek'] = matchweek
    df_gk['date'] = date
    return df_player, df_gk




def get_matchday_data(top, end, linklist):
    pl, gk = frame_for_category('/en/matches/bf52349b', top, end, statsm)
    #pl1, gk1 = frame_for_category('/en/matches/3adf2aa7', top, end, statsm)

    for i in linklist:
        tmppl, tmpgk = frame_for_category(i, top, end, statsm)
        pl = pd.concat([pl, tmppl], axis=0)
        gk = pd.concat([gk, tmpgk], axis=0)
        time.sleep(5)

    return pl, gk

#pl, gk = frame_for_category('/en/matches/51fb894e', 'https://fbref.com/', '', statsm)
#gk

In [86]:
link_list = get_links('https://fbref.com/en/comps/9/10728/schedule/2020-2021-Premier-League-Scores-and-Fixtures')

# split list in 4 parts to not bring down website
list1 = []
list2 = []
list3 = []
list4 = []
count = 0

for i in link_list:
    if count < 100:
       list1.append(i)
    elif count< 200:
        list2.append(i)
    elif count< 300:
        list3.append(i)
    else:
        list4.append(i)
    count = count +1

In [92]:

df_1, gk_1 = get_matchday_data('https://fbref.com/', '', list1 )
df_2, gk_2 = get_matchday_data('https://fbref.com/', '', list2 )
df_3, gk_3 = get_matchday_data('https://fbref.com/', '', list3 )
df_4, gk_4 = get_matchday_data('https://fbref.com/', '', list4 )

#df_test, gk_test = get_matchday_data('https://fbref.com/', '', link_list )

https://fbref.com//en/matches/bf52349b
https://fbref.com//en/matches/be1ee647/Fulham-Wolverhampton-Wanderers-April-9-2021-Premier-League
https://fbref.com//en/matches/ad969519/Manchester-City-Leeds-United-April-10-2021-Premier-League
https://fbref.com//en/matches/20bdebdc/Liverpool-Aston-Villa-April-10-2021-Premier-League
https://fbref.com//en/matches/54b07679/Crystal-Palace-Chelsea-April-10-2021-Premier-League
https://fbref.com//en/matches/daebaaa1/Burnley-Newcastle-United-April-11-2021-Premier-League
https://fbref.com//en/matches/06b2801a/West-Ham-United-Leicester-City-April-11-2021-Premier-League
https://fbref.com//en/matches/a51d567c/Tottenham-Hotspur-Manchester-United-April-11-2021-Premier-League
https://fbref.com//en/matches/34395b54/Sheffield-United-Arsenal-April-11-2021-Premier-League
https://fbref.com//en/matches/3dee3681/West-Bromwich-Albion-Southampton-April-12-2021-Premier-League
https://fbref.com//en/matches/ac6ac8f8/Brighton-and-Hove-Albion-Everton-April-12-2021-Premier-L

In [102]:
df_1

,player,shirtnumber,nationality,position,age,minutes,goals,assists,touches,sca,...,xa,passes,passes_pct,progressive_passes,progressive_carries,dribbles_completed,dribbles,Team,matchweek,date
0,Aboubakar Kamara,47.0,mr MTN,FW,25-189,62.0,0.0,0.0,26.0,0.0,...,0.0,16.0,87.5,0.0,2.0,0.0,2.0,Fulham,(Matchweek 1),2020-09-12
1,Aleksandar Mitrović,9.0,rs SRB,FW,25-362,28.0,0.0,0.0,7.0,0.0,...,0.0,5.0,60.0,0.0,0.0,1.0,1.0,Fulham,(Matchweek 1),2020-09-12
2,Ivan Cavaleiro,17.0,pt POR,LW,26-330,90.0,0.0,0.0,41.0,3.0,...,0.1,35.0,80.0,2.0,4.0,3.0,4.0,Fulham,(Matchweek 1),2020-09-12
3,Neeskens Kebano,7.0,cd COD,RW,28-186,62.0,0.0,0.0,30.0,1.0,...,0.0,23.0,82.6,1.0,3.0,2.0,3.0,Fulham,(Matchweek 1),2020-09-12
4,Andre-Frank Zambo Anguissa,29.0,cm CMR,RW,24-301,28.0,0.0,0.0,23.0,1.0,...,0.0,17.0,94.1,1.0,2.0,1.0,1.0,Fulham,(Matchweek 1),2020-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,Aboubakar Kamara,47.0,mr MTN,FW,25-273,19.0,0.0,0.0,3.0,0.0,...,0.0,1.0,100.0,0.0,1.0,0.0,0.0,Fulham,(Matchweek 11),2020-12-05
10,Tosin Adarabioyo,16.0,eng ENG,CB,23-072,90.0,0.0,0.0,46.0,0.0,...,0.0,31.0,83.9,1.0,0.0,0.0,0.0,Fulham,(Matchweek 11),2020-12-05
11,Joachim Andersen,5.0,dk DEN,CB,24-188,90.0,0.0,0.0,55.0,0.0,...,0.0,45.0,80.0,2.0,1.0,1.0,1.0,Fulham,(Matchweek 11),2020-12-05
12,Ola Aina,34.0,ng NGA,CB,24-058,90.0,0.0,0.0,41.0,0.0,...,0.0,37.0,67.6,4.0,1.0,0.0,0.0,Fulham,(Matchweek 11),2020-12-05


In [94]:
def concatenate_files(f1, f2, f3, f4):
    total = pd.concat([f1, f2, f3, f4], axis=0)
    total = total.drop_duplicates()
    total.reset_index(inplace=True, drop=True)
    return total

gk_2020 = concatenate_files(gk_1, gk_2, gk_3, gk_4)
pl_2020 = concatenate_files(df_1, df_2, df_3, df_4)



In [95]:
pl_2020['matchweek'] = pl_2020['matchweek'].str.slice(start=-3, stop=-1).astype(float)
gk_2020['matchweek'] = gk_2020['matchweek'].str.slice(start=-3, stop=-1).astype(float)
pl_2020['position'] = pl_2020['position'].str.slice(start=0, stop=2)
gk_2020

,player,minutes,age,gk_shots_on_target_against,gk_goals_against,gk_saves,gk_save_pct,gk_psxg,gk_passes_completed_launched,gk_passes,gk_passes_throws,gk_pct_passes_launched,gk_def_actions_outside_pen_area,Team,matchweek,date
0,Marek Rodák,90.0,23-274,5.0,3.0,2.0,40.0,2.5,6.0,22.0,3.0,40.9,1.0,Fulham,1.0,2020-09-12
1,Bernd Leno,90.0,28-192,2.0,0.0,2.0,100.0,0.0,1.0,28.0,2.0,0.0,1.0,Arsenal,1.0,2020-09-12
2,Vicente Guaita,90.0,33-246,5.0,0.0,5.0,100.0,0.8,6.0,20.0,0.0,95.0,0.0,Crystal Palace,1.0,2020-09-12
3,Alex McCarthy,90.0,30-284,3.0,1.0,2.0,66.7,0.7,8.0,44.0,8.0,38.6,4.0,Southampton,1.0,2020-09-12
4,Alisson,90.0,27-346,3.0,3.0,0.0,0.0,2.1,2.0,26.0,5.0,34.6,0.0,Liverpool,1.0,2020-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,Emiliano Martínez,90.0,28-263,6.0,1.0,5.0,83.3,1.2,9.0,24.0,4.0,62.5,0.0,Aston Villa,38.0,2021-05-23
761,Edouard Mendy,45.0,29-083,2.0,1.0,1.0,50.0,1.1,0.0,9.0,4.0,0.0,0.0,Chelsea,38.0,2021-05-23
762,Kepa Arrizabalaga,45.0,26-232,0.0,1.0,0.0,0.0,0.6,2.0,9.0,0.0,33.3,1.0,Chelsea,38.0,2021-05-23
763,Łukasz Fabiański,90.0,36-035,5.0,0.0,5.0,100.0,1.0,14.0,28.0,6.0,60.7,0.0,West Ham United,38.0,2021-05-23


In [96]:
gk_2020['position'] = 'GK'

In [97]:
gk_2020

,player,minutes,age,gk_shots_on_target_against,gk_goals_against,gk_saves,gk_save_pct,gk_psxg,gk_passes_completed_launched,gk_passes,gk_passes_throws,gk_pct_passes_launched,gk_def_actions_outside_pen_area,Team,matchweek,date,position
0,Marek Rodák,90.0,23-274,5.0,3.0,2.0,40.0,2.5,6.0,22.0,3.0,40.9,1.0,Fulham,1.0,2020-09-12,GK
1,Bernd Leno,90.0,28-192,2.0,0.0,2.0,100.0,0.0,1.0,28.0,2.0,0.0,1.0,Arsenal,1.0,2020-09-12,GK
2,Vicente Guaita,90.0,33-246,5.0,0.0,5.0,100.0,0.8,6.0,20.0,0.0,95.0,0.0,Crystal Palace,1.0,2020-09-12,GK
3,Alex McCarthy,90.0,30-284,3.0,1.0,2.0,66.7,0.7,8.0,44.0,8.0,38.6,4.0,Southampton,1.0,2020-09-12,GK
4,Alisson,90.0,27-346,3.0,3.0,0.0,0.0,2.1,2.0,26.0,5.0,34.6,0.0,Liverpool,1.0,2020-09-12,GK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,Emiliano Martínez,90.0,28-263,6.0,1.0,5.0,83.3,1.2,9.0,24.0,4.0,62.5,0.0,Aston Villa,38.0,2021-05-23,GK
761,Edouard Mendy,45.0,29-083,2.0,1.0,1.0,50.0,1.1,0.0,9.0,4.0,0.0,0.0,Chelsea,38.0,2021-05-23,GK
762,Kepa Arrizabalaga,45.0,26-232,0.0,1.0,0.0,0.0,0.6,2.0,9.0,0.0,33.3,1.0,Chelsea,38.0,2021-05-23,GK
763,Łukasz Fabiański,90.0,36-035,5.0,0.0,5.0,100.0,1.0,14.0,28.0,6.0,60.7,0.0,West Ham United,38.0,2021-05-23,GK


In [98]:
#map position to category
def get_category(df):
    df.loc[df['position'] == 'GK', 'category'] = 'Goalkeeper'
    df.loc[df['position'] == 'DF', 'category'] = 'Defender'
    df.loc[df['position'] == 'RB', 'category'] = 'Defender'
    df.loc[df['position'] == 'LB', 'category'] = 'Defender'
    df.loc[df['position'] == 'CB', 'category'] = 'Defender'
    df.loc[df['position'] == 'WB', 'category'] = 'Defender'
    df.loc[df['position'] == 'FB', 'category'] = 'Defender'
    df.loc[df['position'] == 'MF', 'category'] = 'Midfield'
    df.loc[df['position'] == 'DM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'CM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'AM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'LM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'RM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'WM', 'category'] = 'Midfield'
    df.loc[df['position'] == 'FW', 'category'] = 'Forward'
    df.loc[df['position'] == 'RW', 'category'] = 'Forward'
    df.loc[df['position'] == 'LW', 'category'] = 'Forward'
    return df

pl_2020 = get_category(pl_2020)

In [99]:
def create_csv(df):
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    df.to_csv('PL2020_players.csv',index=False)
    return df

create_csv(pl_2020)


,player,shirtnumber,nationality,position,age,minutes,goals,assists,touches,sca,...,passes,passes_pct,progressive_passes,progressive_carries,dribbles_completed,dribbles,Team,matchweek,date,category
0,Aboubakar Kamara,47.0,mr MTN,FW,25-189,62.0,0.0,0.0,26.0,0.0,...,16.0,87.5,0.0,2.0,0.0,2.0,Fulham,1.0,2020-09-12,Forward
1,Aleksandar Mitrović,9.0,rs SRB,FW,25-362,28.0,0.0,0.0,7.0,0.0,...,5.0,60.0,0.0,0.0,1.0,1.0,Fulham,1.0,2020-09-12,Forward
2,Ivan Cavaleiro,17.0,pt POR,LW,26-330,90.0,0.0,0.0,41.0,3.0,...,35.0,80.0,2.0,4.0,3.0,4.0,Fulham,1.0,2020-09-12,Forward
3,Neeskens Kebano,7.0,cd COD,RW,28-186,62.0,0.0,0.0,30.0,1.0,...,23.0,82.6,1.0,3.0,2.0,3.0,Fulham,1.0,2020-09-12,Forward
4,Andre-Frank Zambo Anguissa,29.0,cm CMR,RW,24-301,28.0,0.0,0.0,23.0,1.0,...,17.0,94.1,1.0,2.0,1.0,1.0,Fulham,1.0,2020-09-12,Forward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10389,Jannik Vestergaard,4.0,dk DEN,CB,28-293,90.0,0.0,0.0,70.0,0.0,...,59.0,84.7,5.0,2.0,0.0,1.0,Southampton,38.0,2021-05-23,Defender
10390,Jan Bednarek,35.0,pl POL,CB,25-041,90.0,0.0,0.0,81.0,0.0,...,62.0,83.9,1.0,5.0,0.0,0.0,Southampton,38.0,2021-05-23,Defender
10391,Kyle Walker-Peters,2.0,eng ENG,RB,24-040,59.0,0.0,0.0,56.0,1.0,...,40.0,72.5,0.0,3.0,1.0,1.0,Southampton,38.0,2021-05-23,Defender
10392,Oriol Romeu,6.0,es ESP,RB,29-241,31.0,0.0,0.0,29.0,1.0,...,27.0,88.9,7.0,5.0,0.0,0.0,Southampton,38.0,2021-05-23,Defender


In [100]:
def create_csv(df):
    df = df.drop_duplicates()
    df.reset_index(inplace=True, drop=True)
    df.to_csv('PL2020_keepers.csv',index=False)
    return df

create_csv(gk_2020)


,player,minutes,age,gk_shots_on_target_against,gk_goals_against,gk_saves,gk_save_pct,gk_psxg,gk_passes_completed_launched,gk_passes,gk_passes_throws,gk_pct_passes_launched,gk_def_actions_outside_pen_area,Team,matchweek,date,position
0,Marek Rodák,90.0,23-274,5.0,3.0,2.0,40.0,2.5,6.0,22.0,3.0,40.9,1.0,Fulham,1.0,2020-09-12,GK
1,Bernd Leno,90.0,28-192,2.0,0.0,2.0,100.0,0.0,1.0,28.0,2.0,0.0,1.0,Arsenal,1.0,2020-09-12,GK
2,Vicente Guaita,90.0,33-246,5.0,0.0,5.0,100.0,0.8,6.0,20.0,0.0,95.0,0.0,Crystal Palace,1.0,2020-09-12,GK
3,Alex McCarthy,90.0,30-284,3.0,1.0,2.0,66.7,0.7,8.0,44.0,8.0,38.6,4.0,Southampton,1.0,2020-09-12,GK
4,Alisson,90.0,27-346,3.0,3.0,0.0,0.0,2.1,2.0,26.0,5.0,34.6,0.0,Liverpool,1.0,2020-09-12,GK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760,Emiliano Martínez,90.0,28-263,6.0,1.0,5.0,83.3,1.2,9.0,24.0,4.0,62.5,0.0,Aston Villa,38.0,2021-05-23,GK
761,Edouard Mendy,45.0,29-083,2.0,1.0,1.0,50.0,1.1,0.0,9.0,4.0,0.0,0.0,Chelsea,38.0,2021-05-23,GK
762,Kepa Arrizabalaga,45.0,26-232,0.0,1.0,0.0,0.0,0.6,2.0,9.0,0.0,33.3,1.0,Chelsea,38.0,2021-05-23,GK
763,Łukasz Fabiański,90.0,36-035,5.0,0.0,5.0,100.0,1.0,14.0,28.0,6.0,60.7,0.0,West Ham United,38.0,2021-05-23,GK


In [129]:
#gk_2021[['player', 'matchweek']]

test_group = gk_2021.groupby(['player']).sum()['saves']
gk_strength = test_group.sort_values(ascending=False)
gk_strength

player
Illan Meslier          130.0
David de Gea           122.0
José Sá                119.0
Kasper Schmeichel      113.0
Jordan Pickford        107.0
Nick Pope               98.0
Robert Sánchez          96.0
Łukasz Fabiański        95.0
Tim Krul                93.0
Hugo Lloris             90.0
Emiliano Martínez       84.0
Vicente Guaita          79.0
Aaron Ramsdale          77.0
David Raya              68.0
Edouard Mendy           67.0
Fraser Forster          67.0
Martin Dúbravka         66.0
Alisson                 65.0
Ben Foster              56.0
Ederson                 54.0
Álvaro Fernández        46.0
Daniel Bachmann         39.0
Alex McCarthy           38.0
Angus Gunn              26.0
Karl Darlow             25.0
Kepa Arrizabalaga       11.0
Jack Butland            11.0
Bernd Leno               9.0
Freddie Woodman          9.0
Asmir Begović            7.0
Wayne Hennessey          7.0
Willy Caballero          6.0
Caoimhín Kelleher        5.0
Kristoffer Klaesson      4.0
Jason S

In [133]:
rslt_df = gk_2021[gk_2021['player'] == 'Bernd Leno']
rslt_df[['player', 'matchweek']]

,player,matchweek
1,Bernd Leno,(Matchweek 1)
36,Bernd Leno,(Matchweek 2)
41,Bernd Leno,(Matchweek 3)
573,Bernd Leno,(Matchweek 30)
